In [1]:
import numpy as np

In [2]:
def initialize_neighbors(i, j, h=9, w=9):
    """Create a list of indexes of the neighbors for postion (i,j)
    """
    neighbors_indexes = list()
    # indexes of the neighbors along the vertical line
    for i_ in range(9):
        if i_ != i:
            index = i_*w + j 
            neighbors_indexes.append(index)

    # indexes of the neighbors along the horizontal line
    for j_ in range(9):
        if j_ != j:
            index = i*w + j_ 
            neighbors_indexes.append(index)
    
    i_center, j_center = 3*(i//3) + 1, 3*(j//3) +1
    
    # indexes of the neighbors for the coressponding cube(3*3) with center (i_center, j_center)
    for i_ in range(9):
        for j_ in range(9):
            if abs(i_center - i_) <= 1 and abs(j_center - j_) <= 1:
                index = i_*w + j_
                neighbors_indexes.append(index)
    # delete repeated indexes
    neighbors_indexes = list(set(neighbors_indexes))
    # delete index with coressponding to position (i, j)
    neighbors_indexes.remove(i*w + j)
    return neighbors_indexes


def recalculate_q(i, j, k):
    current_value = q[i, j, k-1]
    # false q can not become positive
    if current_value == False:
        return False 
    # loop through all neighbors
    for index_neighbor, _ in enumerate(neighbors_structure[i, j, :]):
        temp_value = False
        for k_ in K:
            temp_value = temp_value | g[i, j, index_neighbor, k-1, k_-1]
        current_value = current_value & temp_value
    return current_value


def update_step():
    ### q
    for i in range(9):
        for j in range(9):
            if sudoku[i, j]!=0:
                for k in K:
                    q[i, j, k-1] = recalculate_q(i, j, k)

                
    ### g
    for i in range(9):
        for j in range(9):
            for index_neighbor, neighbor in enumerate(neighbors_structure[i, j, :]):
                for k in K:
                    for k_ in K:
                        # restore position (i_, j_) from index
                        i_ = int(neighbor // w)
                        j_ = int(neighbor % w)
                        g[i, j, index_neighbor, k-1, k_-1] = q[i, j, k-1] & q[i_, j_, k_-1]

In [3]:
w, h = 9, 9
neighbors_structure = np.zeros((9, 9, 20)) # h, w, n_neighbors
for i in range(h):
    for j in range(w):
        neighbors_structure[i, j, :] = initialize_neighbors(i, j)

q = np.ones((9, 9, 9), dtype=np.bool) # h, w, n_digits
g = np.ones((9, 9, 20, 9, 9), dtype=np.bool) # h, w, n_neighbors, k, k_
K = [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
# sudoku = np.array([[0,0,2,0,9,0,0,6,0],
#                    [0,4,0,0,0,1,0,0,8],
#                    [0,7,0,4,2,0,0,0,3],
#                    [5,0,0,0,0,0,3,0,0],
#                    [0,0,1,0,6,0,5,0,0],
#                    [0,0,3,0,0,0,0,0,6],
#                    [1,0,0,0,5,7,0,4,0],
#                    [6,0,0,9,0,0,0,2,0],
#                    [0,2,0,0,8,0,1,0,0]], dtype=np.uint8)

sudoku = np.array([[2, 6, 4, 0, 0, 3, 0, 0, 1],
                   [0, 9, 5, 8, 0, 0, 3, 6, 0],
                   [1, 0, 0, 0, 5, 0, 7, 0, 0],
                   [0, 0, 0, 0, 2, 5, 6, 9, 0],
                   [0, 0, 0, 4, 0, 8, 0, 0, 0],
                   [0, 8, 2, 3, 7, 0, 0, 0, 0],
                   [0, 0, 9, 0, 3, 0, 0, 0, 4],
                   [0, 1, 6, 0, 0, 2, 9, 7, 0],
                   [3, 0, 0, 9, 0, 0, 5, 2, 6]])
sudoku.shape

(9, 9)

## initialize q

In [5]:
# if sudoku[i, j]!= 0, then assign for all k(k!=sudoku[i,j]) q[i,j,k]=False
for i in range(9):
    for j in range(9):
        if sudoku[i, j] != 0:
            for k in K:
                if k != sudoku[i, j]:
                    q[i, j, k-1]= False

# if sudoku[i, j]!= 0, then all its neighbors can take value sudoku[i, j]                   
for i in range(9):
    for j in range(9):
        if sudoku[i, j] != 0:
            for neighbor in neighbors_structure[i, j, :]:
                i_ = int(neighbor // w)
                j_ = int(neighbor % w)
                q[i_, j_, sudoku[i, j]-1] = False

## Main loop

In [6]:
for epoch in range(10):
    q_prev, g_prev = q.copy(), g.copy()
    update_step()
    print((q_prev != q).sum())
    print((g_prev != g).sum())

0
126068
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
